## Modelo de Transferencia del Aprendizaje


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [5]:
# Cargar los datos
data = pd.read_csv('productos.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'productos.csv'

In [ ]:
# Separar las características (X) y la variable objetivo (y)
X = data.drop(columns=['Est. Monthly Sales'])
y = data['Est. Monthly Sales']

In [6]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [ ]:
# Entrenar un modelo base (Random Forest en este ejemplo)
base_model = RandomForestRegressor()
base_model.fit(X_train, y_train)

In [7]:
# Evaluar el rendimiento del modelo base
y_pred_base = base_model.predict(X_test)
mse_base = mean_squared_error(y_test, y_pred_base)
print("MSE del modelo base:", mse_base)

# Asumiendo que no tenemos historial de ventas, continuamos con transferencia de aprendizaje
# Recolecta datos de productos similares en el nuevo mercado y guárdalos en un archivo CSV "nuevo_producto.csv"


NameError: name 'base_model' is not defined

In [ ]:
# Cargar datos del nuevo producto
nuevo_producto = pd.read_csv('nuevo_producto.csv')

In [ ]:
# Ajustar el modelo base con los datos del nuevo producto (transferencia de aprendizaje)
base_model.fit(X, y)

In [ ]:
# Hacer predicciones para el nuevo producto
predicciones_nuevo_producto = base_model.predict(nuevo_producto)

In [ ]:
print("Predicciones para el nuevo producto:", predicciones_nuevo_producto)
